In [5]:
import pandas as pd
import xml.etree.ElementTree as ET
import os
import pprint as pp
from zipfile import ZipFile
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
import csv
import ast
import json
import flatten_json
from flatten_json import flatten
from pandas.io.json import json_normalize
import xmltodict
import os
import numpy as np

pip install flatten_json

pip install xmltodict

In [47]:
def listfiles(rootdir):
    f = []
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            path = os.path.join(subdir, file)
            f.append(path)
    return f

In [48]:
#rootdir = '/Users/tnagireddy/Documents/Tasks/XML_toDF_BAT/Files'
rootdir = 'E:/tnagireddy/Files 7 - 8'


f = listfiles(rootdir)
print(len(f))

2


In [49]:
def ques(root):
    questions_q = pd.DataFrame()
    for q in range(len(root[8])):
        tags, attributes, values = [], [], []
        question_id = root[8][q].attrib
        question_id = question_id["QuestionID"]
        question_id = question_id.replace("ID",'')

        for child in root[8][q]:
            tags.append(child.tag)
            attributes.append(question_id)
            values.append(child.text)

            if len(child) > 0:
                for c in child:
                    tags.append(c.tag)
                    attributes.append(c.attrib)
                    if len(c) > 0:
                        for i in c:
                            values.append(i.text)
                    else:
                        values.append(c.text)

            question = pd.DataFrame({
                'tags':tags,
                'attributes':attributes,
                'values':values
            })

        questions_q = questions_q.append(question)
        
    return questions_q

In [50]:
def resp(root):
    tags, values = [], []

    for child in root[0]:
        tags.append(child.tag)
        values.append(child.text)

    responses = pd.DataFrame({
        'tags':tags,
        'values':values
    })
    
    return responses

In [51]:
total_files = []
for file in f:
    zip_file = ZipFile(file, 'r')
    zip_file_contents = zip_file.infolist() # storing folders for specific zip
    total_files.append(len(zip_file_contents))
print(total_files)
print(sum(total_files))

[19723, 25123]
44846


In [53]:
f[0]
zip_file = ZipFile(f[0], 'r')
zip_file_contents = zip_file.infolist()
zip_file_contents[16]

<ZipInfo filename='BAT_Questionnaires_20191023/145739/844f62cf-eaac-4156-a90c-a42960168a5f.xml' compress_type=deflate external_attr=0x20 file_size=1296372 compress_size=120011>

In [54]:
zip_file = ZipFile(f[0], 'r')
zip_file_contents = zip_file.infolist()
zip_file_content=zip_file.read(zip_file_contents[16])
str_data=str(zip_file_content, 'utf-8')
data=ET.fromstring(str_data)
tree=ET.ElementTree(data)
root=tree.getroot()
q = ques(root)

In [55]:
root

<Element 'SurveyDefinition' at 0x0000017909974138>

In [56]:
q = q[q['tags'] == "QuestionText"]

In [57]:
q = q.reset_index()
q = q.drop(['tags','index'],1)
#q = q.transpose()

In [58]:
q.iloc[6,1]

'1.4 I confirm that I have the authority and am authorised to represent and complete this questionnaire on behalf of the company'

q

In [12]:
#pp.pprint(zip_file_contents)
questions=0
responses=0
errors=0
resps = q
resps = resps.rename(columns={'values':'questions'})
total = sum(total_files)
#total = 10
pbar = tqdm(total=total)
error_files = []

for file in f:
    zip_file = ZipFile(file, 'r')
    zip_file_contents = zip_file.infolist()
    total_f = len(zip_file_contents)

    for i in range(total_f):
        try:
            if str(zip_file_contents[i]).split(" ")[1][-4:-1] == 'xml':
                zip_file_content=zip_file.read(zip_file_contents[i])
                str_data=str(zip_file_content, 'utf-8')
                data=ET.fromstring(str_data)
                tree=ET.ElementTree(data)
                root=tree.getroot()
                str_root=str(root).split(" ")[1].strip("'")
                if str_root == "SurveyDefinition":
                    questions+=1
                elif str_root == "xml":
                    responses+=1
                    resps_rename = resp(root)
                    resps_rename = resps_rename.rename(columns={"tags": "attributes"})
                    #resps_rename = resps_rename.transpose()
                    resps = resps.merge(resps_rename, on='attributes', how='left')
                    resps = resps.rename(columns={'values':responses})
                    #resps = resps.append(resps_rename)
            pbar.update()
        except:
            errors+=1
            error_files.append(zip_file_contents[i])

print(len(zip_file_contents))
print("questions: {}".format(questions))
print("responses: {}".format(responses))
print("other: {}".format(total-(questions + responses)))
print("errors: {}".format(errors))

25123
questions: 32195
responses: 1424
other: 11227
errors: 1


In [69]:
resps.iloc[:,36][11]

'N°1109  AVENUE BIAYI/ COMMUNE DE LUBUMBASHI'

In [65]:
resps = resps.transpose()
resps

,0,1,2,3,4,5,6,7,8,9,...,281,282,283,284,285,286,287,288,289,290
attributes,Q293,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q93,Q94,Q95,Q96,Q73,Q74,Q75,Q76,Q77,Q78
questions,"<p><span style=""color:#e74c3c;""><b><u>YOU CAN ...","<span style=""color:#ffffff;""><span style=""back...",<strong>1. Official authorisation</strong>,1.1 Your full name (The name of the individual...,1.2 Your role/position in the organisation,1.3 Your email address,1.4 I confirm that I have the authority and am...,1.5 I confirm that the information provided in...,<strong>2. Third party claims</strong>,2.1 Are you a claimant against BAT?<br />\n<sp...,...,Please provide an explanation as to why this a...,Do the terms and conditions in place include a...,Please provide details of the commission fee a...,Please provide an explanation as to why this a...,20. Has your organisation or any individuals a...,Please provide details of the charges<br />\n<...,21. Do any managerial staff or shareholders ha...,Please provide details of the connection<br />...,21. Do you have any non-contractual connection...,Please provide details of the connection<br />...
1,1,1,1,Nicholas Ouma Odhiambo,Senior Manager RMS Client Service,Nicholas.Odhiambo@nielsen.com,1,1,1,2,...,None,2,None,None,2,None,2,None,None,None
2,1,1,1,KADUYU IMMACULATE,PAYABLE AND TREASURY ACCOUNTANT,ikaduyu@ug.nationmedia.com,1,1,1,2,...,None,2,None,None,2,None,2,None,None,None
3,1,1,1,JASON MUTUGI,KEY ACCOUNT MANAGER,"jason@vajas.com, sales@vajas.com",1,1,1,2,...,None,2,None,None,2,None,2,None,None,None
4,1,1,1,JOSEPH KABUGI KARANJA,CHIEF EXECUTIVE,joe.kabugi@capacityoutsourcing.co.ke,1,1,1,2,...,None,2,None,None,2,None,2,None,None,None
5,1,1,1,DAVE LUNGAHO SIGANGA,MANAGING PARTNER,lungaho@siganga.co.ke/dlsiganga@gmail.com,1,1,1,2,...,None,2,None,None,2,None,2,None,None,None
6,1,1,1,James onyango omondi,technical manager/ director,planetboiler@yahoo.com,1,1,1,2,...,None,2,None,None,2,None,None,None,2,None
7,1,1,1,Caleb Owino,Managing Director,c.owino@fireworks-advertising.com,1,1,1,2,...,None,2,None,None,2,None,2,None,None,None
8,1,1,1,Crispin Mbogo,Managing Director,Crispin.Mbogo@droplex.com,1,1,1,2,...,None,2,None,None,2,None,2,None,None,None


In [14]:
resps.to_csv("results.csv")

In [ ]:
resps.to_excel('results-7.xlsx', encoding='utf8')

In [15]:
error_files

[<ZipInfo filename='BAT_Questionnaires_20191023/145715/90e70b7c-a0c4-44d5-ba69-c7f77ee2b1b6.xml' compress_type=deflate external_attr=0x20 file_size=1296237 compress_size=1273>]